In [19]:
!wget https://www.kaggleusercontent.com/kf/123693371/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..q8HeDL3Ubb2PbfQwwRaB6w.FXW1Yctb8eY1OLlKeNgUjh1w_F5f3gnC93zNP0nKb-SkuEOr-KXnQavsjdNxIGoXDF6a5TQR7BFyec1ZquGT-CKpXvU_rzh9pxW7wYpuLipgBPEdJW3yJ8n2LoZzs2wFDSsroExZrhS_TDhoCyo-tnUr4Bkq7NSACxR2Ja3KdjWe8WUWyVMi36gxe7DEyr0LY8t9KIChw4QqowCVZwrvMQ5k-kT7Mn5O42GeatLGCRJb8h3AOrAu6Lvx_OUMFiZlJncXkJoFlzhLtohzpg9IILZJZvl12KzTztFVlF2ZP50SYuP68opuUfCj41FyrVh8e08AfM2xW85iiZ_hqF2iUVDCh_PVlGND_ylZkCBRWYdKokwr_jJuKN996HVVmZ_XKHukfguE7cnRupt24jvZLemn_cPd7fAlFpqQj-JBf5y5bxklHE2x3FhJpkwiUieQV9KAgCJGwbgBjr0K_PrXMgZNI86ETaNYj8LBr7nYi3uMY_TNQuV4GDunikap551a6KsM48d4BAiqM_IrHa04NfdAEuUyXNtDzmhE_45m3aWiBr9O1sTSzXWp9QrArrEOOW0UHjH85s4jowFlZojVcyM637Y6bgCPkk2BtAvItD9idFUApZKpW3_MpVolWjMWjzlYFyXoPuRnwgMAON9R9v3pR1Z1A5U5s7MN4poiRo76zLFnMIHPTivyCLemyRXk.CgVMAclYNuTiGr4-QyOaYA/res_model.h5

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import librosa
import librosa.display
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import IPython
#import tensorflow.keras.layers as L
import tensorflow as tf
#from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
#from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder,StandardScaler
import re
import itertools
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import os


In [21]:
# @title
model_new = tf.keras.models.load_model("res_model.h5")
model_new.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 2376, 512)         3072      
                                                                 
 batch_normalization (Batch  (None, 2376, 512)         2048      
 Normalization)                                                  
                                                                 
 max_pooling1d (MaxPooling1  (None, 1188, 512)         0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 1188, 512)         1311232   
                                                                 
 batch_normalization_1 (Bat  (None, 1188, 512)         2048      
 chNormalization)                                                
                                                        

In [22]:
def zcr(data,frame_length=2048,hop_length=512):
    zcr=librosa.feature.zero_crossing_rate(y=data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(zcr)
def rmse(data,frame_length=2048,hop_length=512):
    rmse=librosa.feature.rms(y=data,frame_length=frame_length,hop_length=hop_length)
    return np.squeeze(rmse)
def mfcc(data,sr,frame_length=2048,hop_length=512,flatten:bool=True):
    mfcc=librosa.feature.mfcc(y=data,sr=sr)
    return np.squeeze(mfcc.T)if not flatten else np.ravel(mfcc.T)

def extract_features(data,sr,frame_length=2048,hop_length=512):
    result=np.array([])

    result=np.hstack((result,
                      zcr(data,frame_length,hop_length),
                      rmse(data,frame_length,hop_length),
                      mfcc(data,sr,frame_length,hop_length)
                     ))
    return result

In [32]:
path="wav.wav"
data,sr=librosa.load(path,duration=2.5,offset=0.6)


In [33]:
aud=extract_features(data,sr)
aud
aud=[aud, aud, aud, aud, aud]

In [34]:
dfpredict=pd.DataFrame(aud)
dfpredict

,0,1,2,3,4,5,6,7,8,9,...,2366,2367,2368,2369,2370,2371,2372,2373,2374,2375
0,0.052246,0.074219,0.102539,0.097656,0.094238,0.089355,0.07959,0.07666,0.074707,0.087402,...,-22.70079,-19.315344,-3.813098,-1.391756,1.935263,-16.947777,7.346428,-6.165598,5.942199,19.050514
1,0.052246,0.074219,0.102539,0.097656,0.094238,0.089355,0.07959,0.07666,0.074707,0.087402,...,-22.70079,-19.315344,-3.813098,-1.391756,1.935263,-16.947777,7.346428,-6.165598,5.942199,19.050514
2,0.052246,0.074219,0.102539,0.097656,0.094238,0.089355,0.07959,0.07666,0.074707,0.087402,...,-22.70079,-19.315344,-3.813098,-1.391756,1.935263,-16.947777,7.346428,-6.165598,5.942199,19.050514
3,0.052246,0.074219,0.102539,0.097656,0.094238,0.089355,0.07959,0.07666,0.074707,0.087402,...,-22.70079,-19.315344,-3.813098,-1.391756,1.935263,-16.947777,7.346428,-6.165598,5.942199,19.050514
4,0.052246,0.074219,0.102539,0.097656,0.094238,0.089355,0.07959,0.07666,0.074707,0.087402,...,-22.70079,-19.315344,-3.813098,-1.391756,1.935263,-16.947777,7.346428,-6.165598,5.942199,19.050514


In [35]:
dfpredict=dfpredict.fillna(0)
dfpredict

,0,1,2,3,4,5,6,7,8,9,...,2366,2367,2368,2369,2370,2371,2372,2373,2374,2375
0,0.052246,0.074219,0.102539,0.097656,0.094238,0.089355,0.07959,0.07666,0.074707,0.087402,...,-22.70079,-19.315344,-3.813098,-1.391756,1.935263,-16.947777,7.346428,-6.165598,5.942199,19.050514
1,0.052246,0.074219,0.102539,0.097656,0.094238,0.089355,0.07959,0.07666,0.074707,0.087402,...,-22.70079,-19.315344,-3.813098,-1.391756,1.935263,-16.947777,7.346428,-6.165598,5.942199,19.050514
2,0.052246,0.074219,0.102539,0.097656,0.094238,0.089355,0.07959,0.07666,0.074707,0.087402,...,-22.70079,-19.315344,-3.813098,-1.391756,1.935263,-16.947777,7.346428,-6.165598,5.942199,19.050514
3,0.052246,0.074219,0.102539,0.097656,0.094238,0.089355,0.07959,0.07666,0.074707,0.087402,...,-22.70079,-19.315344,-3.813098,-1.391756,1.935263,-16.947777,7.346428,-6.165598,5.942199,19.050514
4,0.052246,0.074219,0.102539,0.097656,0.094238,0.089355,0.07959,0.07666,0.074707,0.087402,...,-22.70079,-19.315344,-3.813098,-1.391756,1.935263,-16.947777,7.346428,-6.165598,5.942199,19.050514


In [36]:
scaler=StandardScaler()
predict_data=scaler.fit_transform(dfpredict)
predict_data
predict_data.shape


(5, 2376)

In [37]:
predict_data=np.expand_dims(predict_data,axis=2)
predict_data
predict_data.shape

(5, 2376, 1)

In [38]:
data_pred = model_new.predict(predict_data)
data_pred


1/1 [==============================] - 0s 306ms/step


array([[4.5249911e-07, 6.9024563e-06, 9.7213483e-01, 6.6039624e-04,
        3.4884222e-05, 2.7161483e-02, 1.0148523e-06],
       [4.5249911e-07, 6.9024563e-06, 9.7213483e-01, 6.6039624e-04,
        3.4884222e-05, 2.7161483e-02, 1.0148523e-06],
       [4.5249911e-07, 6.9024563e-06, 9.7213483e-01, 6.6039624e-04,
        3.4884222e-05, 2.7161483e-02, 1.0148523e-06],
       [4.5249911e-07, 6.9024563e-06, 9.7213483e-01, 6.6039624e-04,
        3.4884222e-05, 2.7161483e-02, 1.0148523e-06],
       [4.5249911e-07, 6.9024563e-06, 9.7213483e-01, 6.6039624e-04,
        3.4884222e-05, 2.7161481e-02, 1.0148523e-06]], dtype=float32)

In [39]:
data_pred = np.argmax(data_pred, axis=1)
data_pred

array([2, 2, 2, 2, 2], dtype=int64)